In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from GaussianDiffusion import GaussianDiffusion
import imageio
import numpy as np
import math

In [ ]:
# Constants
# Values chosen in the paper
beta_start = 0.0001
beta_end = 0.02
time_steps = 1000
clip_min = -1.0
clip_max = 1.0

img_size = 64
img_channels = 3

In [ ]:
#Get the time step functions
diffusion_functions = GaussianDiffusion(beta_start=beta_start, beta_end=beta_end, time_steps=time_steps)

In [ ]:
#Load the model
network = tf.keras.models.load_model("64pxModelsRandomUniform/Epoch80WithAttention")

In [ ]:
def generate_images(num_images = 2):
    all_samples = []
    #Generate the starting point - noisy images
    samples = tf.random.normal(shape=(num_images, img_size, img_size, img_channels), dtype = tf.float32)
    #Incremental sampling process
    for t in reversed(range(0, time_steps)):
        #Get time steps of correct shape
        tt = tf.cast(tf.fill(num_images, t), dtype = tf.int32)
        #Using the time step and samples, predict the noise in the samples
        pred_noise = network([samples, tt])
        #Get the images from the samples + predicted noise
        samples = diffusion_functions.p_sample(pred_noise, samples, tt)
        if t %50 == 0:
            #print(f"Time step: {time_steps - t}")
            pass
        pass
        all_samples.append(samples)
    #Clip the image
    #samples = tf.clip_by_value(samples, clip_value_min=clip_min, clip_value_max=clip_max)

    return all_samples

In [ ]:
def get_all_frames_of_one_image(frames, index):
    relevant_frames = []

    for step, image in enumerate(frames):
        image = image[index]
        image = (image + 1)/2
        image = image*255
        image = np.asarray(image, np.uint8)
        if (step+1) < 800:
            if (step+1)%20 == 0:
                relevant_frames.append(image)
                pass
            pass
        if (step+1) >=800:
            if (step+1)%5 == 0:
                relevant_frames.append(image)
                pass
            pass
        pass
    return relevant_frames

In [37]:
def save_gifs(frames, num_images):
    for i in range(num_images):
        relevant_frames = get_all_frames_of_one_image(frames, i)
        #print(len(relevant_frames))
        # Create the GIF using imageio
        with imageio.get_writer(f'DenoisingGiFs/diffusionGif{i}.gif', mode='I') as writer:
            for image in relevant_frames:
                writer.append_data(image)
                pass
            pass
        pass
    pass

def save_mega_gif(frames, path):
    with imageio.get_writer(path, mode='I') as writer:
        for image in frames:
            writer.append_data(image)
            pass
        pass
    pass

In [16]:
num_images = 16
list_batch_frames = generate_images(num_images)
save_gifs(frames=list_batch_frames, num_images=num_images)

In [42]:
def concatenate_frames_mega(list_batch_frames):
    batch_size, dim, _, channels = list_batch_frames[0].shape
    grid_rows = grid_cols = int(math.sqrt(batch_size))
    list_frames = []
    for step, batch_frames in enumerate(list_batch_frames):
        grid = tf.reshape(batch_frames, (grid_rows, grid_cols, dim, dim, channels))
        grid = tf.transpose(grid, (0, 2, 1, 3, 4))
        large_image = tf.reshape(grid, (grid_rows * dim, grid_cols * dim, channels))
        image = large_image
        image = (image + 1)/2
        image = image*255
        image = np.asarray(image, np.uint8)
        if (step+1) < 800:
            if (step+1)%20 == 0:
                list_frames.append(image)
                pass
            pass
        if (step+1) >=800:
            if (step+1)%5 == 0:
                list_frames.append(image)
                pass
            pass
        pass

        
    return list_frames

In [44]:
concatted_frames = concatenate_frames_mega(list_batch_frames=list_batch_frames)
save_mega_gif(concatted_frames, 'DenoisingGiFs/MegadiffusionGif.gif')

The images which were generated by the diffusion model:

![GiF](DenoisingGiFs/MegadiffusionGif.gif)